In [ ]:
#The goal of this project is to test which model perform better on condition of with or without standardize 
#and whether the algorithms would be affected by dummy variable trap

#The data is the consumers data including gender, age, financial concerning data etc. from a bank
#I want to predict if the consumer would exit or not

In [60]:
import numpy as np
import pandas as pd

data = (r"C:\Users\aoc32\Desktop\workspace\exit_or_not.csv")
df = pd.read_csv(data)
df.head()
#import data

,Unnamed: 0,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,551,15806307,S2336,720,S2,Male,38,5,114051.97,2,0,1,107577.29,0
1,1,6897,15709621,S1500,682,S0,Female,54,4,62397.41,1,1,0,113088.60,1
2,2,4588,15619340,S1865,672,S0,Female,31,5,119903.67,1,1,1,132925.17,0
3,3,291,15620746,S1672,592,S2,Female,40,4,104257.86,1,1,0,110857.33,0
4,4,1673,15646372,S2532,753,S2,Male,42,5,120387.73,1,0,1,126378.57,0


In [61]:
df = df.drop(['Unnamed: 0'],axis=1)
df = df.drop(['RowNumber'],axis=1)
df = df.drop(['CustomerId'],axis=1)
df = df.drop('Surname',axis=1)
#drop noises

In [62]:
dummies = pd.get_dummies(list(df['Geography']))
df_dummies = pd.DataFrame(dummies)
df = df.join(df_dummies)
df = df.drop(['Geography'],axis=1)
df = df.rename(columns={'Gender': 'Male'})
df.loc[df.Male == 'Male' , 'Male'] = 1
df.loc[df.Male == 'Female', 'Male']= 0
#transfer feature to dummies

In [63]:
df_x = df.loc[:, df.columns != 'Exited']
df_y = df.loc[:,df.columns == 'Exited']
df_x.head()

#split data to independent variables and dependent variables

,CreditScore,Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,S0,S1,S2
0,720,1,38,5,114051.97,2,0,1,107577.29,0,0,1
1,682,0,54,4,62397.41,1,1,0,113088.60,1,0,0
2,672,0,31,5,119903.67,1,1,1,132925.17,1,0,0
3,592,0,40,4,104257.86,1,1,0,110857.33,0,0,1
4,753,1,42,5,120387.73,1,0,1,126378.57,0,0,1


In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb

In [65]:
Log = LogisticRegression()
scores = cross_val_score(Log, df_x, df_y, cv=5, scoring='accuracy')
Log_score = scores.mean()

Svc = SVC()
scores = cross_val_score(Svc, df_x, df_y, cv=5, scoring='accuracy')
Svc_score = scores.mean()

Rfc = RandomForestClassifier()
scores = cross_val_score(Rfc, df_x, df_y, cv=5, scoring='accuracy')
Rfc_score = scores.mean()

XG =xgb.XGBClassifier()
scores = cross_val_score(XG, df_x, df_y, cv=5, scoring='accuracy')
XG_score = scores.mean()

nstd_navoid_scores = {'nstd_navoid':pd.Series([Log_score,Svc_score,Rfc_score,XG_score], index=['Log','SVC','RFC','XGb'])}
table1 = pd.DataFrame(nstd_navoid_scores)

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be chan

In [66]:
from sklearn.preprocessing import StandardScaler
df_x[['CreditScore', 'Age','Tenure','Balance','NumOfProducts','EstimatedSalary']] = \
StandardScaler().fit_transform(df_x[['CreditScore', 'Age','Tenure','Balance','NumOfProducts','EstimatedSalary']])

#standardize independent variables which is not dummies

D:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [67]:
df_x.head()

,CreditScore,Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,S0,S1,S2
0,1.509906,1,-0.663721,0.339933,0.428978,0.799546,0,1,0.304927,0,0,1
1,0.866637,0,1.465241,-0.445812,-1.214512,-0.917139,1,0,0.529787,1,0,0
2,0.697356,0,-1.595141,0.339933,0.615161,-0.917139,1,1,1.339115,1,0,0
3,-0.656894,0,-0.397600,-0.445812,0.117359,-0.917139,1,0,0.438752,0,0,1
4,2.068534,1,-0.131480,0.339933,0.630562,-0.917139,0,1,1.072015,0,0,1


In [68]:
Log = LogisticRegression()
scores = cross_val_score(Log, df_x, df_y, cv=5, scoring='accuracy')
Log_score = scores.mean()

Svc = SVC()
scores = cross_val_score(Svc, df_x, df_y, cv=5, scoring='accuracy')
Svc_score = scores.mean()

Rfc = RandomForestClassifier()
scores = cross_val_score(Rfc, df_x, df_y, cv=5, scoring='accuracy')
Rfc_score = scores.mean()

XG =xgb.XGBClassifier()
scores = cross_val_score(XG, df_x, df_y, cv=5, scoring='accuracy')
XG_score = scores.mean()

std_navoid_scores = {'std_navoid':pd.Series([Log_score,Svc_score,Rfc_score,XG_score], index=['Log','SVC','RFC','XGb'])}
table2 = pd.DataFrame(std_navoid_scores)

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be chan

In [69]:
df_x = df_x.drop(['S2'],axis=1)
df_x.head()

#drop S2 to avoid dummy variable trap

,CreditScore,Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,S0,S1
0,1.509906,1,-0.663721,0.339933,0.428978,0.799546,0,1,0.304927,0,0
1,0.866637,0,1.465241,-0.445812,-1.214512,-0.917139,1,0,0.529787,1,0
2,0.697356,0,-1.595141,0.339933,0.615161,-0.917139,1,1,1.339115,1,0
3,-0.656894,0,-0.397600,-0.445812,0.117359,-0.917139,1,0,0.438752,0,0
4,2.068534,1,-0.131480,0.339933,0.630562,-0.917139,0,1,1.072015,0,0


In [70]:
Log = LogisticRegression()
scores = cross_val_score(Log, df_x, df_y, cv=5, scoring='accuracy')
Log_score = scores.mean()

Svc = SVC()
scores = cross_val_score(Svc, df_x, df_y, cv=5, scoring='accuracy')
Svc_score = scores.mean()

Rfc = RandomForestClassifier()
scores = cross_val_score(Rfc, df_x, df_y, cv=5, scoring='accuracy')
Rfc_score = scores.mean()

XG =xgb.XGBClassifier()
scores = cross_val_score(XG, df_x, df_y, cv=5, scoring='accuracy')
XG_score = scores.mean()

std_avoid_scores = {'std_avoid':pd.Series([Log_score,Svc_score,Rfc_score,XG_score], index=['Log','SVC','RFC','XGb'])}
table3 = pd.DataFrame(std_avoid_scores)

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be chan

In [71]:
compare = table1.join(table2['std_navoid']).join(table3['std_avoid'])

In [72]:
print(compare)

     nstd_navoid  std_navoid  std_avoid
Log     0.795375    0.812626   0.812501
SVC     0.796000    0.855373   0.855373
RFC     0.851372    0.851246   0.850122
XGb     0.854373    0.854498   0.850624


In [ ]:
#Conclusion: Standardizing the independent variables make LogisticRegression and SVC better off, 
#             but does not matter much for RandomForest and XGboost. Dummy variable trap does
#             not matter much for LogisticRegression, SVC, and RandomForest, while it affect 
#             XGboost a little. 
#To predict the consumer would exit or not, I will choose SVC with standardized data.